# 理解Embedding层和线性层的区别

PyTorch中的嵌入层实现了与执行矩阵乘法的线性层相同的功能;
我们使用嵌入层的原因是计算效率,我们将使用PyTorch中的代码示例逐步研究这种关系

In [4]:
import torch
import torch.nn as nn
print("PyTorch version:", torch.__version__)

PyTorch version: 2.3.1+cpu


# Embedding层

In [3]:
idx = torch.tensor([2, 3, 1])

num_idx = max(idx) + 1

output_dim = 5

In [5]:
torch.manual_seed(123)

embedding_layers = nn.Embedding(num_idx, output_dim)

In [6]:
embedding_layers.weight

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  1.5810],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015],
        [ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953]], requires_grad=True)

In [8]:
embedding_layers(torch.tensor([1]))

tensor([[ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

可以可视化一下看看发生了什么

![Alt text](../../../img/LLM/ch01/embedding%5B1%5D.png)

In [10]:
embedding_layers(torch.tensor([2]))

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315]],
       grad_fn=<EmbeddingBackward0>)

![Alt text](../../../img/LLM/ch01/embedding%5B2%5D.png)

现在，让我们转换之前定义的所有训练示例：

In [11]:
idx = torch.tensor([2, 3, 1])
embedding_layers(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

![Alt text](../../../img/LLM/ch01/embedding_lookup.png)

# Linear

- 现在，我们将证明上面的嵌入层实现了与nn完全相同的功能。PyTorch中一个热编码表示上的线性层
- 首先，让我们将token ID转换为一个热表示：

In [12]:
onehot = torch.nn.functional.one_hot(idx)
onehot

tensor([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]])

接下来，我们初始化一个线性层，它进行矩阵乘法 XWT

In [13]:
torch.manual_seed(123)
linear = torch.nn.Linear(num_idx, output_dim, bias=False)
linear.weight

Parameter containing:
tensor([[-0.2039,  0.0166, -0.2483,  0.1886],
        [-0.4260,  0.3665, -0.3634, -0.3975],
        [-0.3159,  0.2264, -0.1847,  0.1871],
        [-0.4244, -0.3034, -0.1836, -0.0983],
        [-0.3814,  0.3274, -0.1179,  0.1605]], requires_grad=True)

请注意，PyTorch中的线性层也使用小的随机权重进行初始化；要直接将其与上面的嵌入层进行比较，我们必须使用相同的小随机权重，这就是为什么我们在这里重新分配它们：

In [14]:
linear.weight = torch.nn.Parameter(embedding_layers.weight.T.detach())

现在，我们可以在输入的一个热编码表示上使用线性层：

In [15]:
linear(onehot.float())

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]], grad_fn=<MmBackward0>)

正如我们所看到的，这与我们使用嵌入层时得到的结果完全相同：

In [16]:
embedding_layers(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

第一个训练示例的token ID的以下计算：

![Alt text](../../../img/LLM/ch01/example_first.png)

![Alt text](../../../img/LLM/ch01/example_second.png)

由于每一个热编码行中除一个索引外的所有索引都为0（按设计），因此此矩阵乘法基本上与查找一个热元素相同
在一个热编码上使用矩阵乘法相当于嵌入层查找，但如果我们使用大的嵌入矩阵，则可能效率低下，因为存在大量浪费的乘零运算